# Desafio Calculadora inmobiliaria

In [14]:
import pandas as pd
import geopandas as gpd
import numpy as np
import statsmodels.formula.api as smf
import pysal as ps
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [17]:
df = pd.read_csv("properatti.csv")
df.dropna()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
96,96,sell,apartment,San Bernardo,|Argentina|Buenos Aires Costa Atlántica|San Be...,Argentina,Buenos Aires Costa Atlántica,3429228.0,"-36.6806689,-56.6774832",-36.680669,...,30.0,720.000000,1800.000000,3.0,2.0,800.0,http://www.properati.com.ar/15c0x_venta_depart...,"Departamento de 2 ambientes, ubicado en la cos...",DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/9/GbdfNPqf43VErc...
691,691,sell,PH,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5666711,-58.4720061",-34.566671,...,52.0,2923.076923,2923.076923,6.0,2.0,1800.0,http://www.properati.com.ar/15fqi_venta_ph_bel...,"Venta de Departamento 1 DORM en BELGRANO, Capi...",PH EN VENTA,https://thumbs4.properati.com/7/MEz4_dEg2qrZaX...
692,692,sell,PH,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5666711,-58.4720061",-34.566671,...,52.0,2923.076923,2923.076923,6.0,3.0,1800.0,http://www.properati.com.ar/15fqj_venta_ph_bel...,"Venta de Departamento 1 DORM en BELGRANO, Capi...",PH EN VENTA,https://thumbs4.properati.com/6/FxryMiabZX3EGZ...
693,693,sell,PH,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5666225,-58.4721291",-34.566623,...,52.0,2923.076923,2923.076923,6.0,3.0,1800.0,http://www.properati.com.ar/15fqk_venta_ph_bel...,"Venta de Departamento 1 DORM en BELGRANO, Capi...",PH EN VENTA,https://thumbs4.properati.com/0/fHtkOyhk8sPUhG...
694,694,sell,PH,Coghlan,|Argentina|Capital Federal|Coghlan|,Argentina,Capital Federal,3435359.0,"-34.5666643,-58.471989",-34.566664,...,52.0,2923.076923,2923.076923,6.0,2.0,1800.0,http://www.properati.com.ar/15fql_venta_ph_cog...,"Venta de Departamento 1 DORM en BELGRANO, Capi...",PH EN VENTA,https://thumbs4.properati.com/4/ShIEdJrWC7sVZW...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113200,113200,sell,apartment,Caballito,|Argentina|Capital Federal|Caballito|,Argentina,Capital Federal,3435874.0,"-34.6196508,-58.4425282",-34.619651,...,51.0,2870.370370,3039.215686,8.0,3.0,2500.0,http://www.properati.com.ar/1c81a_venta_depart...,Venta de Departamento 3 AMBIENTES en Caballito...,DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/5/sYQcdM0eoMhvjz...
113201,113201,sell,apartment,Constitución,|Argentina|Capital Federal|Constitución|,Argentina,Capital Federal,3435257.0,"-34.6233888,-58.3843262",-34.623389,...,49.0,1979.591837,1979.591837,9.0,3.0,1386.0,http://www.properati.com.ar/1c81b_venta_depart...,Venta de Departamento 3 AMBIENTES en Constituc...,DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/6/R3wKVNrXBAbi3b...
116148,116148,sell,apartment,Balvanera,|Argentina|Capital Federal|Balvanera|,Argentina,Capital Federal,6693228.0,"-34.5999459,-58.4036457",-34.599946,...,38.0,2842.105263,2842.105263,8.0,1.0,10000.0,http://www.properati.com.ar/1cbyq_venta_depart...,Venta de Departamento MONOAMBIENTE en Balvaner...,DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/7/PLlPKGaOovoxJE...
117105,117105,sell,apartment,Recoleta,|Argentina|Capital Federal|Recoleta|,Argentina,Capital Federal,3429595.0,"-34.5864118,-58.4004266",-34.586412,...,74.0,2972.972973,2972.972973,4.0,4.0,3374.0,http://www.properati.com.ar/1cd6o_venta_depart...,4 AMBIENTES. FRENTE. 74 M2. TODO A NUEVO. BALC...,DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/7/482kgWzc-nBAHd...


In [11]:
gdf = gpd.GeoDataFrame(df)

In [13]:
gdf.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')